<a href="https://colab.research.google.com/github/guptavishu1000/langgraph-crash-course/blob/main/3_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langgraph
!pip install -U langchain-google-genai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.3/153.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.1/56.1 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.7/216.7 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 43.7 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requ

In [2]:
from typing import Annotated
from dotenv import load_dotenv
load_dotenv()

from typing_extensions import TypedDict
from langchain.chat_models import init_chat_model
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

In [8]:
from google.colab import userdata
import os

# Load the secret key into an environment variable
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [9]:
from langchain.chat_models import init_chat_model

In [10]:
help(init_chat_model)

Help on function init_chat_model in module langchain.chat_models.base:

init_chat_model(model: 'Optional[str]' = None, *, model_provider: 'Optional[str]' = None, configurable_fields: "Optional[Union[Literal['any'], list[str], tuple[str, ...]]]" = None, config_prefix: 'Optional[str]' = None, **kwargs: 'Any') -> 'Union[BaseChatModel, _ConfigurableModel]'
    Initialize a ChatModel in a single line using the model's name and provider.

    .. note::
        Must have the integration package corresponding to the model provider installed.
        You should look at the `provider integration's API reference <https://python.langchain.com/api_reference/reference.html#integrations>`__
        to see what parameters are supported by the model.

    Args:
        model: The name of the model, e.g. ``'o3-mini'``, ``'claude-3-5-sonnet-latest'``. You can
            also specify model and model provider in a single argument using
            ``'{model_provider}:{model}'`` format, e.g. ``'openai:o1'`

In [11]:
llm = init_chat_model("google_genai:gemini-2.0-flash")
llm.invoke([{"role": "user", "content": "Who walked on the moon for the first time?"}])

AIMessage(content='Neil Armstrong was the first person to walk on the moon.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--4eebefc8-ad23-4e02-a71b-1853a228e158-0', usage_metadata={'input_tokens': 10, 'output_tokens': 13, 'total_tokens': 23, 'input_token_details': {'cache_read': 0}})

In [12]:
llm = init_chat_model("google_genai:gemini-2.0-flash")

class State(TypedDict):
    messages: Annotated[list, add_messages]

def chatbot(state: State) -> State:
    return {"messages": [llm.invoke(state["messages"])]}

builder = StateGraph(State)
builder.add_node("chatbot_node", chatbot)

builder.add_edge(START, "chatbot_node")
builder.add_edge("chatbot_node", END)

graph = builder.compile()

In [13]:
message = {"role": "user", "content": "Who walked on the moon for the first time? Print only the name"}
# message = {"role": "user", "content": "What is the latest price of MSFT stock?"}
response = graph.invoke({"messages":[message]})

response["messages"]

[HumanMessage(content='Who walked on the moon for the first time? Print only the name', additional_kwargs={}, response_metadata={}, id='65c617d7-4ad9-44f7-8236-3b7e40623153'),
 AIMessage(content='Neil Armstrong', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--16d28ea8-c6aa-4fb8-b38a-d2ca0fcfd0c8-0', usage_metadata={'input_tokens': 14, 'output_tokens': 3, 'total_tokens': 17, 'input_token_details': {'cache_read': 0}})]

In [14]:
state = None
while True:
    in_message = input("You: ")
    if in_message.lower() in {"quit","exit"}:
        break
    if state is None:
        state: State = {
            "messages": [{"role": "user", "content": in_message}]
        }
    else:
        state["messages"].append({"role": "user", "content": in_message})

    state = graph.invoke(state)
    print("Bot:", state["messages"][-1].content)

You: who is the 7th prime minister of india and what do you mean by Statistics
Bot: The 7th Prime Minister of India was **Vishwanath Pratap Singh**, often referred to as **V. P. Singh**. He served from December 2, 1989, to November 10, 1990.

Now, let's talk about **Statistics**:

Statistics is a branch of mathematics that deals with the collection, analysis, interpretation, presentation, and organization of data. It's a powerful tool used to understand and make inferences from data in various fields.

Here's a breakdown of what that means:

*   **Collection:** Gathering data from various sources, such as surveys, experiments, observations, or existing databases.

*   **Organization:** Arranging data in a meaningful way, often through tables, charts, or graphs, to make it easier to understand and analyze.

*   **Presentation:** Displaying data visually or in written form, highlighting key patterns and trends.

*   **Analysis:** Applying mathematical and computational techniques to exam